In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_transforms = transforms.Compose([
    # transforms.Resize(size=(32, 32)),
    transforms.ToTensor()
])
train_dataset = datasets.MNIST(root = "../../data/mnist", train = True, transform=train_transforms, download = True)
test_dataset = datasets.MNIST(root = "../../data/mnist", train = False, transform=train_transforms, download = True)

  Using cached tb_nightly-2.18.0a20240723-py3-none-any.whl.metadata (1.6 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached Markdown-3.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.0.3-py3-none-any.whl.metadata (3.7 kB)
Using cached tb_nightly-2.18.0a20240723-py3-none-any.whl (5.5 MB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
   ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.1 MB 660.6 kB/s eta 0:00:07
   - -------------------------------------- 0.2/4.1 MB 2.4 MB/s eta 0:00:02
   ------ --------------------------------- 0.7/4.1 MB 5.4 MB/s eta 0:00:01
   ------------ --------------------------- 1.3/4.1 MB 7.3 MB/s eta 0:00:01
   ----------------- ---------------------- 1.8/4.1 MB 8.3 MB/s eta 0:00:01
   ----------------------- ---------------- 2.4/4.1 MB 9.0 MB/s eta 0:00:01
   --

In [4]:
class NN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(in_channels, 50)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Hyperparameters
EPOCHS = 2
num_classes = 10
learning_rate = 0.001
batch_size = 64

# RNN parameters
num_layers = 2
input_size = 28
hidden_size = 256
sequence_length = 28

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out


In [7]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out


In [8]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [6]:
class BidirectionalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BidirectionalLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out


In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
model = BidirectionalLSTM(input_size, hidden_size, num_layers, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()


In [9]:
for epoch in range(EPOCHS):
    for batch, (data, label) in enumerate(train_dataloader):
        data = data.to(device).squeeze(1)
        label = label.to(device)
        
        y_pred = model(data)
        loss = loss_fn(y_pred, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        


In [10]:
def check_accuracy(loader, model):


    num_correct = 0
    num_samples = 0
    model.eval()


    with torch.no_grad():
        for x, y in loader:

            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()

            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_dataloader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_dataloader, model)*100:.2f}")

Accuracy on training set: 97.86
Accuracy on test set: 97.59


: 